In [ ]:
import requests
import pandas as pd

# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/?utm_source=chatgpt.com
API_KEY = "cwyQbMvXDASP9PPj75JoG4kZFlNThjpKSgsNBc4q"
BASE = "https://developer.nrel.gov/api/alt-fuel-stations/v1/nearest.json"

def fetch_ev_in_radius(lat, lon, radius_mi=50):
    params = {
        "api_key": API_KEY,
        "latitude": lat,          # 必填
        "longitude": lon,         # 必填
        "radius": radius_mi,      # 0-500，文档允许
        "fuel_type": "ELEC",
        "access": "public",
        "status": "E",
        "country": "US",
        "limit": "all"            # 一次性取全量（radius 不是 infinite 时允许）
        # 注意：不要传 ev_charging_level="" 这种空值；若要筛 DC 快充再加： "ev_charging_level": "dc_fast"
    }
    r = requests.get(BASE, params=params, timeout=30)
    try:
        r.raise_for_status()
    except requests.HTTPError:
        # 打印服务端返回的错误信息，便于定位
        print("Request URL:", r.url)
        print("Response text:", r.text)
        raise
    return r.json().get("fuel_stations", [])

def to_dataframe(rows):
    # 展开常用字段；可按需扩展
    def pick(x, k): 
        return x.get(k) if isinstance(x, dict) else None
    recs = []
    for s in rows:
        recs.append({
            "id": s.get("id"),
            "station_name": s.get("station_name"),
            "ev_network": s.get("ev_network"),
            "ev_connector_types": ",".join(s.get("ev_connector_types") or []),
            "ev_dc_fast_num": s.get("ev_dc_fast_num"),
            "ev_level2_evse_num": s.get("ev_level2_evse_num"),
            "ev_pricing": s.get("ev_pricing"),
            "city": s.get("city"),
            "state": s.get("state"),
            "zip": s.get("zip"),
            "address": s.get("street_address"),
            "access_days_time": s.get("access_days_time"),
            "status_code": s.get("status_code"),
            "latitude": s.get("latitude"),
            "longitude": s.get("longitude"),
            "updated_at": s.get("updated_at"),
        })
    return pd.DataFrame(recs)

if __name__ == "__main__":
    # 奥斯汀市中心（Texas State Capitol 附近）
    austin_lat, austin_lon = 30.2672, -97.7431
    rows = fetch_ev_in_radius(austin_lat, austin_lon, radius_mi=50)
    df = to_dataframe(rows)
    df.to_csv("./data/raw data/charging-station-set.csv", index=False, encoding="utf-8")
    print(f"Saved {len(df)} stations to ./data/raw data/charging-station-set.csv")
